In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd

df_pc = pd.read_parquet("/allen/aics/modeling/ritvik/pcna/manifest.parquet")
df_orig = pd.read_csv("/allen/aics/assay-dev/computational/data/4DN_handoff_Apr2022_testing/PCNA_manifest_for_suraj_with_brightfield.csv")
# df_sample = df_pc.loc[df_pc['structure_name'].isin(['PCNA'])]
df_sample = df_pc.groupby('structure_name').apply(lambda x: x.sample(2)).reset_index(drop=True)

In [2]:
from src.features.diagnosticsheet import diagnostic_sheet
import warnings
import matplotlib.pyplot as plt
import numpy as np

plt.style.use("dark_background")
warnings.simplefilter(action='ignore', category=FutureWarning)

def custom_contrast_adjust(img):
    img = np.where(img < 10000, img, 0)
    img = np.where(img > 0, img, 0)
    return img

diagnostic_sheet(df_sample, 
                 save_dir = "./", # Created a diagnostic_sheets folder in the current working dir
                 image_column = "registered_path", # Pass in the 3D image path or one of the 2D image paths like max_projection_x
                 max_cells=20, # max cells per sheet
                #  channels = [6,7,8], # DNA, Membrane, Structure intensity channels
                #  colors = [[0, 1, 1], [1, 0, 1], [1,1,1]], # Cyan, Magenta, White
                 channels = [6, 8], # DNA, Membrane, Structure intensity channels
                 colors = [[0, 1, 1], [1,1,1]], # Cyan, Magenta, White
                 proj_method = "max", # options - max, mean, sum
                #  metadata = "edge_flag", # Optional, Metadata to stratify the diagnostic sheets
                #  feature = "cell_height", # Optional, Feature to add as text,
                 fig_width = None, # Default is number of columns * 7
                 fig_height = None, # Default is number of rows * 5,
                 distributed_executor_address = None, # An optional executor address to pass to some computation engine.
                 batch_size = None, # process all at once
                contrast_adjust = custom_contrast_adjust,
                 overwrite=True)

/home/ritvik.vasan/anaconda3/envs/test/lib/python3.10/site-packages/pydantic/_migration.py:281: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


(2, 114, 359, 398) max project
(2, 95, 507, 485) max project


PosixUPath('diagnostic_sheets/manifest.csv')